In [1]:
import os
import pandas as pd

In [2]:
data_path = r"C:\Projects\connecteddatahub\data\cleaned_dataframes"
years = ["1999","2000", "2002","2005","2007","2008","2009","2010","2011","2013","2018",]
# years = ['1999']
data_path = r"C:\Projects\connecteddatahub\data"

map_df = pd.read_csv(r"C:\Projects\connecteddatahub\data\maps\cleaned_affiliation.csv")
map_df['Institution'] = map_df['FullName']

In [4]:
system_map = map_df.set_index('AffiliationId')['SystemId']

# build lowercase institution map for case‐insensitive lookup
inst_map_ci = {inst.lower(): sid for inst, sid in zip(map_df['Institution'], map_df['SystemId'])}

for year in years:
    full_df = pd.read_csv(os.path.join(data_path, 'gpt_dataframes', f'cleaned_leadership{year}.csv'))

    # map via AffiliationId first
    aff_map = dict(zip(map_df['AffiliationId'], map_df['SystemId']))
    full_df['SystemId'] = full_df['AffiliationId'].map(aff_map)

    # for any missing SystemId, map via lowercase Institution
    mask = full_df['SystemId'].isna()
    full_df.loc[mask, 'SystemId'] = (
        full_df.loc[mask, 'Institution']
               .str.lower()
               .map(inst_map_ci)
    )

    full_df.to_csv(os.path.join(data_path, 'gpt_dataframes', f'{year}_gptDataframe.csv'), index=False)
